In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2
import os
from zipfile import ZipFile
import time
from datetime import datetime
import itertools
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D, Flatten, MaxPooling2D, Input
from tensorflow.keras import utils
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.image import img_to_array, load_img



np.random.seed(42)
tf.random.set_seed(42)

In [15]:
BASE_DIR = 'UTKFace'
age_labels = []
image_paths = []

image_filenames = os.listdir(BASE_DIR)
random.shuffle(image_filenames)

for image in tqdm(image_filenames):
  image_path = os.path.join(BASE_DIR, image)
  img_components = image.split('_')
  age_label = int(img_components[0])


  age_labels.append(age_label)
  image_paths.append(image_path)

  0%|          | 0/23708 [00:00<?, ?it/s]

In [16]:
df = pd.DataFrame()
df['image_path'], df['age'] = image_paths, age_labels
df.head(5)

,image_path,age
0,UTKFace\26_0_1_20170116205301942.jpg.chip.jpg,26
1,UTKFace\36_0_4_20170117203459599.jpg.chip.jpg,36
2,UTKFace\25_1_1_20170112230805931.jpg.chip.jpg,25
3,UTKFace\37_0_3_20170119194851635.jpg.chip.jpg,37
4,UTKFace\32_1_1_20170113012612272.jpg.chip.jpg,32


In [17]:
df.head()

,image_path,age
0,UTKFace\26_0_1_20170116205301942.jpg.chip.jpg,26
1,UTKFace\36_0_4_20170117203459599.jpg.chip.jpg,36
2,UTKFace\25_1_1_20170112230805931.jpg.chip.jpg,25
3,UTKFace\37_0_3_20170119194851635.jpg.chip.jpg,37
4,UTKFace\32_1_1_20170113012612272.jpg.chip.jpg,32


In [18]:
def class_labels_reassign(age):
    if 1<= age <= 2:
        return 0
    elif 3 <= age <= 9:
        return 1
    elif 10<= age <= 20:
        return 2
    elif 21<= age <= 27:
        return 3
    elif 28 <= age <= 45:
        return 4
    elif 46 <= age <= 65:
        return 5
    else:
        return 6

In [19]:
df["target"] = df["age"].map(class_labels_reassign)

In [20]:
df.head()

,image_path,age,target
0,UTKFace\26_0_1_20170116205301942.jpg.chip.jpg,26,3
1,UTKFace\36_0_4_20170117203459599.jpg.chip.jpg,36,4
2,UTKFace\25_1_1_20170112230805931.jpg.chip.jpg,25,3
3,UTKFace\37_0_3_20170119194851635.jpg.chip.jpg,37,4
4,UTKFace\32_1_1_20170113012612272.jpg.chip.jpg,32,4


In [21]:
train_aug_filenames_list = list(df['image_path'])
train_aug_labels_list = list(df['target'])

In [22]:
train_aug_filenames_tenson = tf.constant(train_aug_filenames_list)
train_aug_labels_tenson = tf.constant(train_aug_labels_list)

In [23]:
num_classes = 7
def _parse_function(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.io.decode_jpeg(image_string, channels = 1)
    label = tf.one_hot(label, num_classes)
    return image_decoded, label

In [24]:
train_aug_dataset = tf.data.Dataset.from_tensor_slices((train_aug_filenames_tenson, train_aug_labels_tenson))
train_aug_dataset = train_aug_dataset.map(_parse_function)
train_aug_dataset = train_aug_dataset.batch(512)

In [25]:

final_cnn = Sequential([
    Input(shape=(200, 200, 1)),  
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(1)  
])

In [26]:
final_cnn.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])

In [27]:
final_cnn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 198, 198, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 99, 99, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 97, 97, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 48, 48, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 46, 46, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 23, 23, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 128)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 109,313 (427.00 KB)

 Trainable params: 109,313 (427.00 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:

def load_images(image_paths, target_size=(200, 200)):
    images = []
    for img_path in image_paths:
        img = load_img(img_path, target_size=target_size, color_mode='grayscale')
        img = img_to_array(img)
        img /= 255.0 
        images.append(img)
    return np.array(images)

image_paths = df['image_path'].values
ages = df['age'].values


train_images = load_images(image_paths)

In [30]:

history = final_cnn.fit(train_images, ages, epochs=10, validation_split=0.2)

Epoch 1/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 521s 875ms/step - loss: 541.5865 - mae: 17.7993 - val_loss: 385.3116 - val_mae: 15.4500
Epoch 2/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 516s 870ms/step - loss: 395.1757 - mae: 15.4867 - val_loss: 385.4159 - val_mae: 15.5280
Epoch 3/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 501s 845ms/step - loss: 393.9864 - mae: 15.4665 - val_loss: 385.0941 - val_mae: 15.5590
Epoch 4/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 502s 847ms/step - loss: 391.4824 - mae: 15.4092 - val_loss: 370.3248 - val_mae: 15.0707
Epoch 5/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 499s 842ms/step - loss: 373.7189 - mae: 14.9032 - val_loss: 359.6163 - val_mae: 14.8039
Epoch 6/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 506s 852ms/step - loss: 362.7002 - mae: 14.6478 - val_loss: 349.5701 - val_mae: 14.7040
Epoch 7/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 506s 853ms/step - loss: 348.6010 - mae: 14.3899 - val_loss: 330.8027 - val_mae: 14.2030
Epoch 8/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 497s 838ms/step - loss: 333.2179 - mae: 14.0809 - val_loss: 315.

In [34]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
def load_and_prepare_image(image_path):

    img = load_img(image_path, target_size=(200, 200), color_mode='grayscale')

    img = img_to_array(img)

    img = img / 255.0

    img = np.expand_dims(img, axis=0)
    return img

image_path = 'woman.png'


img = load_and_prepare_image(image_path)

predicted_age = final_cnn.predict(img)
print(f"Vârsta estimată este: {predicted_age[0][0]} ani")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
Vârsta estimată este: 19.323923110961914 ani


In [35]:

model_save_path = '/mnt/data/final_cnn_model.h5'  
final_cnn.save(model_save_path)
print("Modelul a fost salvat cu succes.")

Modelul a fost salvat cu succes.


In [36]:
from tensorflow.keras.models import load_model


loaded_model = load_model(model_save_path)
print("Modelul a fost încărcat cu succes.")

Modelul a fost încărcat cu succes.


In [37]:

img = load_and_prepare_image('woman.png')  
predicted_age = loaded_model.predict(img)
print(f"Vârsta estimată este: {predicted_age[0][0]} ani")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Vârsta estimată este: 19.323923110961914 ani


In [40]:

model_save_path = 'C:/Users/cosmi/OneDrive/Desktop/an4/licenta/AgeGuessr/Models/final_cnn_model.h5'  
final_cnn.save(model_save_path)
print("Modelul a fost salvat cu succes.")

Modelul a fost salvat cu succes.


In [41]:

img = load_and_prepare_image('woman.png')  
predicted_age = loaded_model.predict(img)
print(f"Vârsta estimată este: {predicted_age[0][0]} ani")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Vârsta estimată este: 19.323923110961914 ani
